# 深度学习计算
:label:`chap_computation`

除了庞大的数据集和强大的硬件，
优秀的软件工具在深度学习的快速发展中发挥了不可或缺的作用。
从2007年发布的开创性的Theano库开始，
灵活的开源工具使研究人员能够快速开发模型原型，
避免了我们使用标准组件时的重复工作，
同时仍然保持了我们进行底层修改的能力。
随着时间的推移，深度学习库已经演变成提供越来越粗糙的抽象。
就像半导体设计师从指定晶体管到逻辑电路再到编写代码一样，
神经网络研究人员已经从考虑单个人工神经元的行为转变为从层的角度构思网络，
通常在设计架构时考虑的是更粗糙的块（block）。

之前我们已经介绍了一些基本的机器学习概念，
并慢慢介绍了功能齐全的深度学习模型。
在上一章中，我们从零开始实现了多层感知机的每个组件，
然后展示了如何利用高级API轻松地实现相同的模型。
为了易于学习，我们调用了深度学习库，但是跳过了它们工作的细节。
在本章中，我们将深入探索深度学习计算的关键组件，
即模型构建、参数访问与初始化、设计自定义层和块、将模型读写到磁盘，
以及利用GPU实现显著的加速。
这些知识将使读者从深度学习“基础用户”变为“高级用户”。
虽然本章不介绍任何新的模型或数据集，
但后面的高级模型章节在很大程度上依赖于本章的知识。

:begin_tab:toc
 - [model-construction](model-construction.ipynb)
 - [parameters](parameters.ipynb)
 - [deferred-init](deferred-init.ipynb)
 - [custom-layer](custom-layer.ipynb)
 - [read-write](read-write.ipynb)
 - [use-gpu](use-gpu.ipynb)
:end_tab:


In [5]:
import torch
import torch.nn as nn

# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
# 定义一个简单的神经网络模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(1000, 1000)  # 输入和输出都设置为 1000

    def forward(self, x):
        x = self.fc(x)
        return x

# 创建模型实例并移动到 GPU
model = SimpleModel().to(device)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# 定义输入数据
input_data = torch.randn(32, 1000).to(device)

# 训练循环
for epoch in range(100000):  # 这里设置迭代次数为10，你可以根据需要调整
    # 前向传播
    output = model(input_data)

    # 计算损失
    loss = criterion(output, torch.randn_like(output).to(device))

    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 打印显存使用情况
    memory_usage = torch.cuda.memory_allocated(device)
    print(f"Epoch {epoch + 1}, 显存使用情况: {memory_usage / (1024 ** 2)} MiB")


    # 释放显存
    torch.cuda.empty_cache()


cuda
Epoch 1, 显存使用情况: 8.00390625 MiB
Epoch 2, 显存使用情况: 8.00390625 MiB
Epoch 3, 显存使用情况: 8.00390625 MiB
Epoch 4, 显存使用情况: 8.00390625 MiB
Epoch 5, 显存使用情况: 8.00390625 MiB
Epoch 6, 显存使用情况: 8.00390625 MiB
Epoch 7, 显存使用情况: 8.00390625 MiB
Epoch 8, 显存使用情况: 8.00390625 MiB
Epoch 9, 显存使用情况: 8.00390625 MiB
Epoch 10, 显存使用情况: 8.00390625 MiB
Epoch 11, 显存使用情况: 8.00390625 MiB
Epoch 12, 显存使用情况: 8.00390625 MiB
Epoch 13, 显存使用情况: 8.00390625 MiB
Epoch 14, 显存使用情况: 8.00390625 MiB
Epoch 15, 显存使用情况: 8.00390625 MiB
Epoch 16, 显存使用情况: 8.00390625 MiB
Epoch 17, 显存使用情况: 8.00390625 MiB
Epoch 18, 显存使用情况: 8.00390625 MiB
Epoch 19, 显存使用情况: 8.00390625 MiB
Epoch 20, 显存使用情况: 8.00390625 MiB
Epoch 21, 显存使用情况: 8.00390625 MiB
Epoch 22, 显存使用情况: 8.00390625 MiB
Epoch 23, 显存使用情况: 8.00390625 MiB
Epoch 24, 显存使用情况: 8.00390625 MiB
Epoch 25, 显存使用情况: 8.00390625 MiB
Epoch 26, 显存使用情况: 8.00390625 MiB
Epoch 27, 显存使用情况: 8.00390625 MiB
Epoch 28, 显存使用情况: 8.00390625 MiB
Epoch 29, 显存使用情况: 8.00390625 MiB
Epoch 30, 显存使用情况: 8.00390625 MiB
Epoch 31, 显存使用

KeyboardInterrupt: 